This attempt was discarded as the pre processed data turned out to be faulty.

# Parallel Inter-Image k-Means Clustering (IIkMC)
## Replication of Han & Lee (2024) using Python + GPU Acceleration
- Preprocessing with CPU parallelism (joblib)
- Clustering with GPU acceleration (CuPy)
- Performance comparison with serial + threaded + GPU

In [ ]:
# Install necessary packages
!pip install rasterio joblib cupy-cuda11x

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 60.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.0/100.0 MB 8.3 MB/s eta 0:00:00


In [ ]:
import numpy as np
import rasterio
import os
from concurrent.futures import ThreadPoolExecutor

In [ ]:
# Mount Google Drive if using large datasets
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Dataset path (adjust this to your drive folder)
dataset_path = '/content/drive/MyDrive/PDC/dataset'

## Parallel Preprocessing (CPU - Threaded)

In [ ]:
def load_landsat_bands_with_mask(scene_folder):
    band_suffixes = ['SR_B2.TIF', 'SR_B3.TIF', 'SR_B4.TIF',
                     'SR_B5.TIF', 'SR_B6.TIF', 'SR_B7.TIF']
    bands = []

    for suffix in band_suffixes:
        band_path = [f for f in os.listdir(scene_folder) if suffix in f][0]
        with rasterio.open(os.path.join(scene_folder, band_path)) as src:
            band = src.read(1).astype(np.float32)
            bands.append(band)

    # Stack into (rows, cols, 6)
    stacked = np.stack(bands, axis=-1)

    # Load QA_AEROSOL band
    qa_path = [f for f in os.listdir(scene_folder) if 'SR_QA_AEROSOL' in f][0]
    with rasterio.open(os.path.join(scene_folder, qa_path)) as src:
        qa = src.read(1)

    # Mask out cloud shadow (bit 3) and cloud (bit 5)
    cloud_shadow = (qa & (1 << 3)) > 0
    cloud = (qa & (1 << 5)) > 0
    bad_mask = cloud_shadow | cloud

    stacked[bad_mask] = np.nan
    return stacked


In [ ]:
def flatten_scene(scene_path):
    img = load_landsat_bands_with_mask(scene_path)
    img_flat = img.reshape(-1, 6)
    img_flat = img_flat[~np.isnan(img_flat).any(axis=1)]
    return img_flat


In [ ]:
def prepare_all_scenes_threaded(dataset_dir, max_workers=4):
    scene_dirs = [os.path.join(dataset_dir, f) for f in os.listdir(dataset_dir)
                  if os.path.isdir(os.path.join(dataset_dir, f))]

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        results = list(executor.map(flatten_scene, scene_dirs))

    return np.vstack(results)


In [ ]:

import time
start = time.time()

X = prepare_all_scenes_threaded(dataset_path, max_workers=3)

end = time.time()
print("Threaded preprocessing complete.")
print("Combined dataset shape:", X.shape)
print(f"Time taken: {end - start:.2f} seconds")

Threaded preprocessing complete.
Combined dataset shape: (115436984, 6)
Time taken: 76.35 seconds


In [ ]:
parallel_time3= end-start
print(parallel_time3)

50.965222120285034


For comparison purposes, checking time with max workers 2

In [ ]:

import time
start = time.time()

X2 = prepare_all_scenes_threaded(dataset_path, max_workers=2)

end = time.time()
print("Threaded preprocessing complete.")
print("Combined dataset shape:", X.shape)
print(f"Time taken: {end - start:.2f} seconds")

Threaded preprocessing complete.
Combined dataset shape: (115436984, 6)
Time taken: 53.84 seconds


In [ ]:
parallel_time2= end - start
print(parallel_time2)

53.841261863708496


In [ ]:
start = time.time()

X1 = prepare_all_scenes_threaded(dataset_path, max_workers=1)

end = time.time()
parallel_time1= end - start
print(parallel_time1)

64.43632245063782


In [ ]:
speedup = parallel_time1/parallel_time3
print(speedup)

1.2487981210690158


The best performance was observed with max_workers=3, achieving a 1.25× speedup over the single-threaded baseline. This matches the expected behavior from the paper, where parallel performance improves until memory and I/O constraints begin to dominate. Beyond 3 threads, the Colab VM exceeds available RAM and fails, which sets an upper bound for practical parallelism in this environment

In [ ]:
!pip install tqdm
from tqdm.notebook import tqdm


Serial computation

In [ ]:
def prepare_all_scenes_serial(dataset_dir):
    scene_dirs = [os.path.join(dataset_dir, f) for f in os.listdir(dataset_dir)
                  if os.path.isdir(os.path.join(dataset_dir, f))]

    all_pixels = []

    for scene_path in tqdm(scene_dirs, desc="Processing scenes (serial)"):
        img_flat = flatten_scene(scene_path)
        all_pixels.append(img_flat)

    return np.vstack(all_pixels)


In [ ]:
import time

start = time.time()
X_serial = prepare_all_scenes_serial(dataset_path)
end = time.time()

print("Serial preprocessing complete.")
print("Combined shape:", X_serial.shape)
print(f"Time taken: {end - start:.2f} seconds")
serialtime= end - start


Processing scenes (serial):   0%|          | 0/5 [00:00<?, ?it/s]

Serial preprocessing complete.
Combined shape: (115436984, 6)
Time taken: 58.78 seconds


Calculating speedup

In [ ]:
serialtime=58.78
speedup = serialtime/parallel_time3
print(speedup)

1.1533355012418272


X is:

A giant pixel matrix

Each row = one valid, non-cloudy pixel from any image

Each column = one spectral band (B2–B7)

## IIkMC Clustering CPU based

In [ ]:
def initialize_centers_min_max(X, k):
    B = X.shape[1]  # Number of bands = 6
    centers = np.zeros((k, B), dtype=np.float32)

    for b in range(B):  # For each band
        min_val = np.min(X[:, b])
        max_val = np.max(X[:, b])
        step = (max_val - min_val) / k
        for i in range(k):
            centers[i, b] = min_val + (i + 0.5) * step  # midpoint of each interval

    return centers


In [ ]:
# Assume X is ready and centers initialized
k = 8
centers = initialize_centers_min_max(X, k)

import time
start = time.time()

labels, final_centers = iikmc_cpu_memory_safe(X, centers, max_iter=10)

end = time.time()
print(f"Memory-safe CPU IIkMC finished in {end - start:.2f} seconds")


In [ ]:
def iikmc_cpu(X, centers, max_iter=10, tol=0.01):
    N, B = X.shape
    k = centers.shape[0]
    labels = np.full(N, -1, dtype=np.int32)

    for iteration in range(max_iter):
        distances = np.linalg.norm(X[:, None, :] - centers[None, :, :], axis=2)
        new_labels = np.argmin(distances, axis=1)

        n_changed = np.sum(labels != new_labels)
        if iteration > 0 and n_changed / N < tol:
            print(f"Converged after {iteration} iterations.")
            break

        labels = new_labels

        for j in range(k):
            cluster_points = X[labels == j]
            if len(cluster_points) > 0:
                centers[j] = np.mean(cluster_points, axis=0)

        print(f"Iteration {iteration+1}: {n_changed} pixels changed")

    return labels, centers


In [ ]:
import time
start = time.time()

labels, final_centers = iikmc_cpu(X, centers, max_iter=10)

end = time.time()
print(f"CPU IIkMC finished in {end - start:.2f} seconds")


## Performance Comparison Table

In [ ]:
# Add serial, parallel, and GPU times
# Plot or print speedup ratios

Chunk based


In [ ]:
!pip install tqdm
import numpy as np
from tqdm.notebook import tqdm
from concurrent.futures import ThreadPoolExecutor


In [ ]:
def initialize_centers_min_max(X, k):
    B = X.shape[1]
    centers = np.zeros((k, B), dtype=np.float32)

    for b in range(B):
        min_val = np.min(X[:, b])
        max_val = np.max(X[:, b])
        step = (max_val - min_val) / k
        for i in range(k):
            centers[i, b] = min_val + (i + 0.5) * step

    return centers


In [ ]:
def iikmc_chunked_parallel(X, centers, max_iter=10, tol=0.01, num_workers=4):
    N, B = X.shape
    k = centers.shape[0]
    labels = np.full(N, -1, dtype=np.int32)
    chunks = np.array_split(np.arange(N), num_workers)

    for iteration in range(max_iter):
        new_labels = np.empty(N, dtype=np.int32)

        def assign_chunk(chunk):
            local_labels = np.empty_like(chunk)
            for idx, i in enumerate(chunk):
                min_dist = float('inf')
                min_j = -1
                for j in range(k):
                    dist = np.linalg.norm(X[i] - centers[j])
                    if dist < min_dist:
                        min_dist = dist
                        min_j = j
                local_labels[idx] = min_j
            return chunk, local_labels

        with ThreadPoolExecutor(max_workers=num_workers) as executor:
            futures = list(tqdm(executor.map(assign_chunk, chunks), total=len(chunks), desc=f"Iteration {iteration+1} - Assigning"))

        for chunk, local_labels in futures:
            new_labels[chunk] = local_labels

        n_changed = np.sum(labels != new_labels)
        if iteration > 0 and n_changed / N < tol:
            print(f"✅ Converged after {iteration} iterations.")
            break
        labels = new_labels

        center_sums = np.zeros((k, B), dtype=np.float32)
        counts = np.zeros(k, dtype=np.int32)

        for i in tqdm(range(N), desc="Updating centers"):
            cluster_id = labels[i]
            center_sums[cluster_id] += X[i]
            counts[cluster_id] += 1

        for j in range(k):
            if counts[j] > 0:
                centers[j] = center_sums[j] / counts[j]

        print(f"Iteration {iteration+1} done: {n_changed} pixels changed.")

    return labels, centers


In [ ]:
k = 8
centers = initialize_centers_min_max(X, k)

import time
start = time.time()
labels, final_centers = iikmc_chunked_parallel(X, centers, max_iter=5, num_workers=4)
end = time.time()

print(f"✅ Chunked Parallel CPU IIkMC finished in {end - start:.2f} seconds")


Iteration 1 - Assigning:   0%|          | 0/4 [00:00<?, ?it/s]

Updating centers:   0%|          | 0/115436984 [00:00<?, ?it/s]

Iteration 1 done: 115436984 pixels changed.


Iteration 2 - Assigning:   0%|          | 0/4 [00:00<?, ?it/s]

Updating centers:   0%|          | 0/115436984 [00:00<?, ?it/s]

Iteration 2 done: 1536787 pixels changed.


Iteration 3 - Assigning:   0%|          | 0/4 [00:00<?, ?it/s]

✅ Converged after 2 iterations.
✅ Chunked Parallel CPU IIkMC finished in 25429.43 seconds


Iteration 1 - Assigning: 100%
 2/2 [2:22:22<00:00, 3518.55s/it]
Updating centers: 100%
 115436984/115436984 [04:32<00:00, 278482.67it/s]
Iteration 1 done: 115436984 pixels changed.
Iteration 2 - Assigning: 100%
 2/2 [2:21:59<00:00, 8519.84s/it]
Updating centers: 100%
 115436984/115436984 [04:32<00:00, 516263.61it/s]
Iteration 2 done: 1536787 pixels changed.
Iteration 3 - Assigning:   0%
 0/2 [00:00<?, ?it/s]
✅ Converged after 2 iterations.
✅ Chunked Parallel CPU IIkMC for two threads finished in 26140.08 seconds